<a href="https://colab.research.google.com/github/heet25itachi/TIMEPASS_BUDDY/blob/main/QuantLib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing a library that is not in Colaboratory

To import a library that's not in Colaboratory by default, you can use `!pip install` or `!apt-get install`.

In [6]:
pip install QuantLib

In [7]:
!pip install matplotlib-venn

In [8]:
!apt-get -qq install -y libfluidsynth1

E: Package 'libfluidsynth1' has no installation candidate


# Install 7zip reader [libarchive](https://pypi.python.org/pypi/libarchive)

In [9]:
# https://pypi.python.org/pypi/libarchive
!apt-get -qq install -y libarchive-dev && pip install -U libarchive
import libarchive

Selecting previously unselected package libarchive-dev:amd64.
(Reading database ... 126455 files and directories currently installed.)
Preparing to unpack .../libarchive-dev_3.6.0-1ubuntu1.5_amd64.deb ...
Unpacking libarchive-dev:amd64 (3.6.0-1ubuntu1.5) ...
Setting up libarchive-dev:amd64 (3.6.0-1ubuntu1.5) ...
Processing triggers for man-db (2.10.2-1) ...
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 11.2 MB/s eta 0:00:00
  Created wheel for libarchive: filename=libarchive-0.4.7-py3-none-any.whl size=31629 sha256=d6fbf901d9085d9bb81ba81c91b679fb57842041c997a43be41af06212d6b3e9
  Stored in directory: /root/.cache/pip/wheels/29/20/ab/f101da7b245b996aa097685ef742243725ea6150f5b3b6d9ed
Successfully built libarchive


# Install GraphViz & [PyDot](https://pypi.python.org/pypi/pydot)

In [10]:
# https://pypi.python.org/pypi/pydot
!apt-get -qq install -y graphviz && pip install pydot
import pydot

# Install [cartopy](http://scitools.org.uk/cartopy/docs/latest/)

In [11]:
!pip install cartopy
import cartopy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 84.5 MB/s eta 0:00:00


In [12]:
# ---
# jupyter:
#   jupytext:
#     formats: py:light
#     text_representation:
#       extension: .py
#       format_name: light
#       format_version: '1.5'
#       jupytext_version: 1.4.2
#   kernelspec:
#     display_name: Python 3
#     language: python
#     name: python3
# ---

# # Bonds
#
# Copyright (&copy;) 2008 Florent Grenier
# Copyright (&copy;) 2010 Lluis Pujol Bajador
#
# This file is part of QuantLib, a free-software/open-source library
# for financial quantitative analysts and developers - https://www.quantlib.org/
#
# QuantLib is free software: you can redistribute it and/or modify it
# under the terms of the QuantLib license.  You should have received a
# # copy of the license along with this program; if not, please email
# <quantlib-dev@lists.sf.net>. The license is also available online at
# <https://www.quantlib.org/license.shtml>.
#
# This program is distributed in the hope that it will be useful, but WITHOUT
# ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS
# FOR A PARTICULAR PURPOSE.  See the license for more details.

#    This example shows how to set up a term structure and then price
#    some simple bonds. The last part is dedicated to peripherical
#    computations such as "Yield to Price" or "Price to Yield"

import QuantLib as ql
import pandas as pd

interactive = 'get_ipython' in globals()

# ### Global data

calendar = ql.TARGET()
settlementDate = ql.Date(18, ql.September, 2008)
settlementDate = calendar.adjust(settlementDate)

fixingDays = 3
settlementDays = 3

todaysDate = calendar.advance(settlementDate, -fixingDays, ql.Days)
ql.Settings.instance().evaluationDate = todaysDate

print("Today: " + str(todaysDate))
print("Settlement Date: " + str(settlementDate))

# ### Market quotes

zcQuotes = [(0.0096, ql.Period(3, ql.Months)), (0.0145, ql.Period(6, ql.Months)), (0.0194, ql.Period(1, ql.Years))]

zcBondsDayCounter = ql.Actual365Fixed()

zcHelpers = [
    ql.DepositRateHelper(
        ql.makeQuoteHandle(r), tenor, fixingDays, calendar, ql.ModifiedFollowing, True, zcBondsDayCounter
    )
    for (r, tenor) in zcQuotes
]

# ### Setup bonds

redemption = 100.0
numberOfBonds = 5

bondQuotes = [
    (ql.Date(15, ql.March, 2005), ql.Date(31, ql.August, 2010), 0.02375, 100.390625),
    (ql.Date(15, ql.June, 2005), ql.Date(31, ql.August, 2011), 0.04625, 106.21875),
    (ql.Date(30, ql.June, 2006), ql.Date(31, ql.August, 2013), 0.03125, 100.59375),
    (ql.Date(15, ql.November, 2002), ql.Date(15, ql.August, 2018), 0.04000, 101.6875),
    (ql.Date(15, ql.May, 1987), ql.Date(15, ql.May, 2038), 0.04500, 102.140625),
]

# ### Definition of the rate helpers

bondsHelpers = []

for issueDate, maturity, couponRate, marketQuote in bondQuotes:
    schedule = ql.Schedule(
        issueDate,
        maturity,
        ql.Period(ql.Semiannual),
        ql.UnitedStates(ql.UnitedStates.GovernmentBond),
        ql.Unadjusted,
        ql.Unadjusted,
        ql.DateGeneration.Backward,
        False,
    )
    bondsHelpers.append(
        ql.FixedRateBondHelper(
            ql.makeQuoteHandle(marketQuote),
            settlementDays,
            100.0,
            schedule,
            [couponRate],
            ql.ActualActual(ql.ActualActual.Bond),
            ql.Unadjusted,
            redemption,
            issueDate,
        )
    )

# ###  Curve building

termStructureDayCounter = ql.ActualActual(ql.ActualActual.ISDA)

bondInstruments = zcHelpers + bondsHelpers

bondDiscountingTermStructure = ql.PiecewiseFlatForward(settlementDate, bondInstruments, termStructureDayCounter)

# ### Building of the LIBOR forecasting curve

dQuotes = [
    (0.043375, ql.Period(1, ql.Weeks)),
    (0.031875, ql.Period(1, ql.Months)),
    (0.0320375, ql.Period(3, ql.Months)),
    (0.03385, ql.Period(6, ql.Months)),
    (0.0338125, ql.Period(9, ql.Months)),
    (0.0335125, ql.Period(1, ql.Years)),
]
sQuotes = [
    (0.0295, ql.Period(2, ql.Years)),
    (0.0323, ql.Period(3, ql.Years)),
    (0.0359, ql.Period(5, ql.Years)),
    (0.0412, ql.Period(10, ql.Years)),
    (0.0433, ql.Period(15, ql.Years)),
]

depositDayCounter = ql.Actual360()
depositHelpers = [
    ql.DepositRateHelper(
        ql.makeQuoteHandle(rate), tenor, fixingDays, calendar, ql.ModifiedFollowing, True, depositDayCounter
    )
    for rate, tenor in dQuotes
]

swFixedLegFrequency = ql.Annual
swFixedLegConvention = ql.Unadjusted
swFixedLegDayCounter = ql.Thirty360(ql.Thirty360.European)
swFloatingLegIndex = ql.Euribor6M()
forwardStart = ql.Period(1, ql.Days)
swapHelpers = [
    ql.SwapRateHelper(
        ql.makeQuoteHandle(rate),
        tenor,
        calendar,
        swFixedLegFrequency,
        swFixedLegConvention,
        swFixedLegDayCounter,
        swFloatingLegIndex,
        ql.QuoteHandle(),
        forwardStart,
    )
    for rate, tenor in sQuotes
]

depoSwapInstruments = depositHelpers + swapHelpers

depoSwapTermStructure = ql.PiecewiseFlatForward(settlementDate, depoSwapInstruments, termStructureDayCounter)

# ### Pricing
#
# Term structures that will be used for pricing:
# the one used for discounting cash flows...

discountingTermStructure = ql.RelinkableYieldTermStructureHandle()

# ...and the one used for forward rate forecasting.

forecastingTermStructure = ql.RelinkableYieldTermStructureHandle()

# Bonds to be priced:

faceAmount = 100

bondEngine = ql.DiscountingBondEngine(discountingTermStructure)

# a zero coupon bond...

zeroCouponBond = ql.ZeroCouponBond(
    settlementDays,
    ql.UnitedStates(ql.UnitedStates.GovernmentBond),
    faceAmount,
    ql.Date(15, ql.August, 2013),
    ql.Following,
    116.92,
    ql.Date(15, ql.August, 2003),
)

zeroCouponBond.setPricingEngine(bondEngine)

# ...a fixed 4.5% US Treasury note...

fixedBondSchedule = ql.Schedule(
    ql.Date(15, ql.May, 2007),
    ql.Date(15, ql.May, 2017),
    ql.Period(ql.Semiannual),
    ql.UnitedStates(ql.UnitedStates.GovernmentBond),
    ql.Unadjusted,
    ql.Unadjusted,
    ql.DateGeneration.Backward,
    False,
)

fixedRateBond = ql.FixedRateBond(
    settlementDays,
    faceAmount,
    fixedBondSchedule,
    [0.045],
    ql.ActualActual(ql.ActualActual.Bond),
    ql.ModifiedFollowing,
    100.0,
    ql.Date(15, ql.May, 2007),
)

fixedRateBond.setPricingEngine(bondEngine)

# ...and a floating rate bond paying 3M USD Libor + 0.1%
# (should and will be priced on another curve later).

liborTermStructure = ql.RelinkableYieldTermStructureHandle()

libor3m = ql.USDLibor(ql.Period(3, ql.Months), liborTermStructure)
libor3m.addFixing(ql.Date(17, ql.April, 2008), 0.028175)
libor3m.addFixing(ql.Date(17, ql.July, 2008), 0.0278625)

floatingBondSchedule = ql.Schedule(
    ql.Date(21, ql.October, 2005),
    ql.Date(21, ql.October, 2010),
    ql.Period(ql.Quarterly),
    ql.UnitedStates(ql.UnitedStates.NYSE),
    ql.Unadjusted,
    ql.Unadjusted,
    ql.DateGeneration.Backward,
    True,
)

floatingRateBond = ql.FloatingRateBond(
    settlementDays,
    faceAmount,
    floatingBondSchedule,
    libor3m,
    ql.Actual360(),
    ql.ModifiedFollowing,
    spreads=[0.001],
    issueDate=ql.Date(21, ql.October, 2005),
)

floatingRateBond.setPricingEngine(bondEngine)

forecastingTermStructure.linkTo(depoSwapTermStructure)
discountingTermStructure.linkTo(bondDiscountingTermStructure)

liborTermStructure.linkTo(depoSwapTermStructure)

# +
data = []
data.append(
    (zeroCouponBond.cleanPrice(), fixedRateBond.cleanPrice(), floatingRateBond.cleanPrice())
)
data.append(
    (zeroCouponBond.dirtyPrice(), fixedRateBond.dirtyPrice(), floatingRateBond.dirtyPrice())
)
data.append(
    (zeroCouponBond.accruedAmount(),
     fixedRateBond.accruedAmount(),
     floatingRateBond.accruedAmount())
)
data.append(
    (None, fixedRateBond.previousCouponRate(), floatingRateBond.previousCouponRate())
)
data.append(
    (None, fixedRateBond.nextCouponRate(), floatingRateBond.nextCouponRate())
)
data.append(
    (zeroCouponBond.bondYield(ql.Actual360(), ql.Compounded, ql.Annual),
     fixedRateBond.bondYield(ql.Actual360(), ql.Compounded, ql.Annual),
     floatingRateBond.bondYield(ql.Actual360(), ql.Compounded, ql.Annual))
)

df = pd.DataFrame(data, columns=["ZC", "Fixed", "Floating"],
                  index=["Clean price", "Dirty price", "Accrued coupon",
                         "Previous coupon rate", "Next coupon rate", "Yield"])
if not interactive:
    print(df)
df
# -

# A few other computations:

# Yield to clean price:

floatingRateBond.cleanPrice(
    floatingRateBond.bondYield(ql.Actual360(), ql.Compounded, ql.Annual),
    ql.Actual360(),
    ql.Compounded,
    ql.Annual,
    settlementDate,
)

# Clean price to yield:

floatingRateBond.bondYield(
    ql.BondPrice(
        floatingRateBond.cleanPrice(),
        ql.BondPrice.Clean,
    ),
    ql.Actual360(),
    ql.Compounded,
    ql.Annual,
    settlementDate,
)


Today: September 15th, 2008
Settlement Date: September 18th, 2008


0.022007488506948527

In [13]:
# ---
# jupyter:
#   jupytext:
#     formats: py:light
#     text_representation:
#       extension: .py
#       format_name: light
#       format_version: '1.5'
#       jupytext_version: 1.4.2
#   kernelspec:
#     display_name: Python 3
#     language: python
#     name: python3
# ---

# # Bermudan swaptions
#
# Copyright (&copy;) 2004, 2005, 2006, 2007 StatPro Italia srl
#
# This file is part of QuantLib, a free-software/open-source library
# for financial quantitative analysts and developers - https://www.quantlib.org/
#
# QuantLib is free software: you can redistribute it and/or modify it under the
# terms of the QuantLib license.  You should have received a copy of the
# license along with this program; if not, please email
# <quantlib-dev@lists.sf.net>. The license is also available online at
# <https://www.quantlib.org/license.shtml>.
#
# This program is distributed in the hope that it will be useful, but WITHOUT
# ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS
# FOR A PARTICULAR PURPOSE.  See the license for more details.

import QuantLib as ql
import pandas as pd

# ### Setup

todaysDate = ql.Date(15, ql.February, 2002)
ql.Settings.instance().evaluationDate = todaysDate
calendar = ql.TARGET()
settlementDate = ql.Date(19, ql.February, 2002)


def calibrate(model, helpers, l, name):
    print("Model: %s" % name)

    method = ql.Simplex(l)
    model.calibrate(helpers, method, ql.EndCriteria(1000, 250, 1e-7, 1e-7, 1e-7))

    print("Parameters: %s" % model.params())

    totalError = 0.0
    data = []
    for swaption, helper in zip(swaptionVols, helpers):
        maturity, length, vol = swaption
        NPV = helper.modelValue()
        implied = helper.impliedVolatility(NPV, 1.0e-4, 1000, 0.05, 0.50)
        error = implied - vol
        totalError += abs(error)
        data.append((maturity, length, vol, implied, error))
    averageError = totalError / len(helpers)

    print(pd.DataFrame(data, columns=["maturity", "length", "volatility", "implied", "error"]))

    print("Average error: %.4f" % averageError)


# ### Market data

swaptionVols = [
    # maturity,          length,             volatility
    (ql.Period(1, ql.Years), ql.Period(5, ql.Years), 0.1148),
    (ql.Period(2, ql.Years), ql.Period(4, ql.Years), 0.1108),
    (ql.Period(3, ql.Years), ql.Period(3, ql.Years), 0.1070),
    (ql.Period(4, ql.Years), ql.Period(2, ql.Years), 0.1021),
    (ql.Period(5, ql.Years), ql.Period(1, ql.Years), 0.1000),
]

# This is a flat yield term structure implying a 1x5 swap at 5%.

rate = ql.makeQuoteHandle(0.04875825)
termStructure = ql.YieldTermStructureHandle(ql.FlatForward(settlementDate, rate, ql.Actual365Fixed()))

# Define the ATM/OTM/ITM swaps:

swapEngine = ql.DiscountingSwapEngine(termStructure)

fixedLegFrequency = ql.Annual
fixedLegTenor = ql.Period(1, ql.Years)
fixedLegConvention = ql.Unadjusted
floatingLegConvention = ql.ModifiedFollowing
fixedLegDayCounter = ql.Thirty360(ql.Thirty360.European)
floatingLegFrequency = ql.Semiannual
floatingLegTenor = ql.Period(6, ql.Months)

payFixed = ql.Swap.Payer
fixingDays = 2
index = ql.Euribor6M(termStructure)
floatingLegDayCounter = index.dayCounter()

swapStart = calendar.advance(settlementDate, 1, ql.Years, floatingLegConvention)
swapEnd = calendar.advance(swapStart, 5, ql.Years, floatingLegConvention)

fixedSchedule = ql.Schedule(
    swapStart,
    swapEnd,
    fixedLegTenor,
    calendar,
    fixedLegConvention,
    fixedLegConvention,
    ql.DateGeneration.Forward,
    False,
)
floatingSchedule = ql.Schedule(
    swapStart,
    swapEnd,
    floatingLegTenor,
    calendar,
    floatingLegConvention,
    floatingLegConvention,
    ql.DateGeneration.Forward,
    False,
)

dummy = ql.VanillaSwap(
    payFixed, 100.0, fixedSchedule, 0.0, fixedLegDayCounter, floatingSchedule, index, 0.0, floatingLegDayCounter
)
dummy.setPricingEngine(swapEngine)
atmRate = dummy.fairRate()

atmSwap = ql.VanillaSwap(
    payFixed, 1000.0, fixedSchedule, atmRate, fixedLegDayCounter,
    floatingSchedule, index, 0.0, floatingLegDayCounter
)

otmSwap = ql.VanillaSwap(
    payFixed, 1000.0, fixedSchedule, atmRate * 1.2, fixedLegDayCounter,
    floatingSchedule, index, 0.0, floatingLegDayCounter
)

itmSwap = ql.VanillaSwap(
    payFixed, 1000.0, fixedSchedule, atmRate * 0.8, fixedLegDayCounter,
    floatingSchedule, index, 0.0, floatingLegDayCounter
)

atmSwap.setPricingEngine(swapEngine)
otmSwap.setPricingEngine(swapEngine)
itmSwap.setPricingEngine(swapEngine)

helpers = [
    ql.SwaptionHelper(
        maturity,
        length,
        ql.makeQuoteHandle(vol),
        index,
        index.tenor(),
        index.dayCounter(),
        index.dayCounter(),
        termStructure,
    )
    for maturity, length, vol in swaptionVols
]

times = {}
for h in helpers:
    for t in h.times():
        times[t] = 1
times = sorted(times.keys())

grid = ql.TimeGrid(times, 30)

G2model = ql.G2(termStructure)
HWmodel = ql.HullWhite(termStructure)
HWmodel2 = ql.HullWhite(termStructure)
BKmodel = ql.BlackKarasinski(termStructure)

# ### Calibrations

for h in helpers:
    h.setPricingEngine(ql.G2SwaptionEngine(G2model, 6.0, 16))
calibrate(G2model, helpers, 0.05, "G2 (analytic formulae)")

for h in helpers:
    h.setPricingEngine(ql.JamshidianSwaptionEngine(HWmodel))
calibrate(HWmodel, helpers, 0.05, "Hull-White (analytic formulae)")

for h in helpers:
    h.setPricingEngine(ql.TreeSwaptionEngine(HWmodel2, grid))
calibrate(HWmodel2, helpers, 0.05, "Hull-White (numerical calibration)")

for h in helpers:
    h.setPricingEngine(ql.TreeSwaptionEngine(BKmodel, grid))
calibrate(BKmodel, helpers, 0.05, "Black-Karasinski (numerical calibration)")


# ### Price Bermudan swaptions on defined swaps

bermudanDates = [d for d in fixedSchedule][:-1]
exercise = ql.BermudanExercise(bermudanDates)

atmSwaption = ql.Swaption(atmSwap, exercise)
otmSwaption = ql.Swaption(otmSwap, exercise)
itmSwaption = ql.Swaption(itmSwap, exercise)

data = []

# +
atmSwaption.setPricingEngine(ql.TreeSwaptionEngine(G2model, 50))
otmSwaption.setPricingEngine(ql.TreeSwaptionEngine(G2model, 50))
itmSwaption.setPricingEngine(ql.TreeSwaptionEngine(G2model, 50))

data.append(("G2 analytic", itmSwaption.NPV(), atmSwaption.NPV(), otmSwaption.NPV()))

# +
atmSwaption.setPricingEngine(ql.TreeSwaptionEngine(HWmodel, 50))
otmSwaption.setPricingEngine(ql.TreeSwaptionEngine(HWmodel, 50))
itmSwaption.setPricingEngine(ql.TreeSwaptionEngine(HWmodel, 50))

data.append(("HW analytic", itmSwaption.NPV(), atmSwaption.NPV(), otmSwaption.NPV()))

# +
atmSwaption.setPricingEngine(ql.TreeSwaptionEngine(HWmodel2, 50))
otmSwaption.setPricingEngine(ql.TreeSwaptionEngine(HWmodel2, 50))
itmSwaption.setPricingEngine(ql.TreeSwaptionEngine(HWmodel2, 50))

data.append(("HW numerical", itmSwaption.NPV(), atmSwaption.NPV(), otmSwaption.NPV()))

# +
atmSwaption.setPricingEngine(ql.TreeSwaptionEngine(BKmodel, 50))
otmSwaption.setPricingEngine(ql.TreeSwaptionEngine(BKmodel, 50))
itmSwaption.setPricingEngine(ql.TreeSwaptionEngine(BKmodel, 50))

data.append(("BK numerical", itmSwaption.NPV(), atmSwaption.NPV(), otmSwaption.NPV()))
# -

print(pd.DataFrame(data, columns=["model", "in-the-money", "at-the-money", "out-of-the-money"]))

Model: G2 (analytic formulae)
Parameters: [ 0.0500893; 0.0015269; 0.0500896; 0.00753992; -0.73451 ]
  maturity length  volatility   implied     error
0       1Y     5Y      0.1148  0.100455 -0.014345
1       2Y     4Y      0.1108  0.105123 -0.005677
2       3Y     3Y      0.1070  0.107050  0.000050
3       4Y     2Y      0.1021  0.108382  0.006282
4       5Y     1Y      0.1000  0.109439  0.009439
Average error: 0.0072
Model: Hull-White (analytic formulae)
Parameters: [ 0.0464152; 0.00586931 ]
  maturity length  volatility   implied     error
0       1Y     5Y      0.1148  0.106204 -0.008596
1       2Y     4Y      0.1108  0.106296 -0.004504
2       3Y     3Y      0.1070  0.106341 -0.000659
3       4Y     2Y      0.1021  0.106443  0.004343
4       5Y     1Y      0.1000  0.106613  0.006613
Average error: 0.0049
Model: Hull-White (numerical calibration)
Parameters: [ 0.0559683; 0.00609935 ]
  maturity length  volatility   implied     error
0       1Y     5Y      0.1148  0.102928 -0.011872


In [14]:
# ---
# jupyter:
#   jupytext:
#     formats: py:light
#     text_representation:
#       extension: .py
#       format_name: light
#       format_version: '1.5'
#       jupytext_version: 1.4.2
#   kernelspec:
#     display_name: Python 3
#     language: python
#     name: python3
# ---

# # Swing options
#
# Copyright (&copy;) 2018 Klaus Spanderen
#
# This file is part of QuantLib, a free-software/open-source library
# for financial quantitative analysts and developers - https://www.quantlib.org/
#
# QuantLib is free software: you can redistribute it and/or modify it under the
# terms of the QuantLib license.  You should have received a copy of the
# license along with this program; if not, please email
# <quantlib-dev@lists.sf.net>. The license is also available online at
# <https://www.quantlib.org/license.shtml>.
#
# This program is distributed in the hope that it will be useful, but WITHOUT
# ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS
# FOR A PARTICULAR PURPOSE.  See the license for more details.

import QuantLib as ql
import math

todaysDate = ql.Date(30, ql.September, 2018)
ql.Settings.instance().evaluationDate = todaysDate
settlementDate = todaysDate
riskFreeRate = ql.FlatForward(settlementDate, 0.0, ql.Actual365Fixed())
dividendYield = ql.FlatForward(settlementDate, 0.0, ql.Actual365Fixed())
underlying = ql.SimpleQuote(30.0)
volatility = ql.BlackConstantVol(todaysDate, ql.TARGET(), 0.20, ql.Actual365Fixed())


exerciseDates = [ql.Date(1, ql.January, 2019) + i for i in range(31)]

swingOption = ql.VanillaSwingOption(
    ql.VanillaForwardPayoff(ql.Option.Call, underlying.value()), ql.SwingExercise(exerciseDates), 0, len(exerciseDates)
)

bsProcess = ql.BlackScholesMertonProcess(
    ql.QuoteHandle(underlying),
    ql.YieldTermStructureHandle(dividendYield),
    ql.YieldTermStructureHandle(riskFreeRate),
    ql.BlackVolTermStructureHandle(volatility),
)

swingOption.setPricingEngine(ql.FdSimpleBSSwingEngine(bsProcess))

print("Black Scholes Price: %f" % swingOption.NPV())

x0 = 0.0
x1 = 0.0

beta = 4.0
eta = 4.0
jumpIntensity = 1.0
speed = 1.0
volatility = 0.1

curveShape = []
for d in exerciseDates:
    t = ql.Actual365Fixed().yearFraction(todaysDate, d)
    gs = (
        math.log(underlying.value())
        - volatility * volatility / (4 * speed) * (1 - math.exp(-2 * speed * t))
        - jumpIntensity / beta * math.log((eta - math.exp(-beta * t)) / (eta - 1.0))
    )
    curveShape.append((t, gs))

ouProcess = ql.ExtendedOrnsteinUhlenbeckProcess(speed, volatility, x0, lambda x: x0)
jProcess = ql.ExtOUWithJumpsProcess(ouProcess, x1, beta, jumpIntensity, eta)

swingOption.setPricingEngine(ql.FdSimpleExtOUJumpSwingEngine(jProcess, riskFreeRate, 25, 25, 200, curveShape))

print("Kluge Model Price  : %f" % swingOption.NPV())

Black Scholes Price: 40.308978
Kluge Model Price  : 40.022780
